In [1]:
import pandas as pd

In [6]:
!ls -l

total 16664
-rw-r--r--   1 apichart  staff     3108 Oct 22 14:14 concat_results.ipynb
-rw-r--r--   1 apichart  staff   107859 Oct 19 09:00 model_parameters.csv
-rw-r--r--   1 apichart  staff   742094 Oct 16 09:02 prediction_store_11_18_dept_1_100.csv
-rw-r--r--   1 apichart  staff   745533 Oct 17 06:23 prediction_store_18_25_dept_1_100.csv
-rwxr-xr-x@  1 apichart  staff    25191 Oct 22 09:10 prediction_store_1_2_dept_1_21.csv
-rwxr-xr-x@  1 apichart  staff    52255 Oct 22 09:10 prediction_store_1_2_dept_21_76.csv
-rwxr-xr-x@  1 apichart  staff    25491 Oct 22 09:10 prediction_store_1_2_dept_77_100.csv
-rw-r--r--   1 apichart  staff  2055232 Oct 19 09:00 prediction_store_25_46_dept_1_100.csv
-rwxr-xr-x@  1 apichart  staff    25097 Oct 22 09:10 prediction_store_2_3_dept_1_21.csv
-rwxr-xr-x@  1 apichart  staff    36966 Oct 22 09:10 prediction_store_2_3_dept_21_51.csv
-rwxr-xr-x@  1 apichart  staff    41704 Oct 22 09:10 prediction_store_2_3_dept_51_100.csv
-rwxr-xr-x@  1 apichart  staff   

## Load result files and concat

In [30]:
results_names = [
'prediction_store_1_2_dept_1_21.csv',
'prediction_store_1_2_dept_21_76.csv',
'prediction_store_1_2_dept_77_100.csv',
'prediction_store_2_3_dept_1_21.csv',
'prediction_store_2_3_dept_21_51.csv',
'prediction_store_2_3_dept_51_100.csv',
'prediction_store_3_4_dept_1_51.csv',
'prediction_store_3_4_dept_51_100.csv',
'prediction_store_4_5_dept_1_51.csv',
'prediction_store_4_5_dept_51_100.csv',
'prediction_store_5_6_dept_1_100.csv',
'prediction_store_6_7_dept_1_100.csv',
'prediction_store_7_9_dept_1_100.csv',
'prediction_store_9_11_dept_1_100.csv',
'prediction_store_11_18_dept_1_100.csv',
'prediction_store_18_25_dept_1_100.csv',
'prediction_store_25_46_dept_1_100.csv'
]

In [31]:
df_all = pd.DataFrame()
for fname in results_names:
    df = pd.read_csv(fname)
    df_all = pd.concat([df_all,df])

In [53]:
df_all = df_all.reset_index(drop=True)

In [54]:
df_all.shape

(129909, 4)

In [56]:
df_all.tail()

,Date,Prediction,Store,Dept
129904,2013-06-28,1829.142279,45,98
129905,2013-07-05,1846.225115,45,98
129906,2013-07-12,1867.852846,45,98
129907,2013-07-19,1530.992450,45,98
129908,2013-07-26,1500.058850,45,98


## Load the test file

In [35]:
df_test = pd.read_csv('walmart-recruiting-store-sales-forecasting/test.csv')

In [36]:
df_test.shape

(115064, 4)

In [63]:
df_test.tail()

,Store,Dept,Date,IsHoliday
115059,45,98,2013-06-28,False
115060,45,98,2013-07-05,False
115061,45,98,2013-07-12,False
115062,45,98,2013-07-19,False
115063,45,98,2013-07-26,False


## Merge results with the test data frame
    - make sure we report only the results required in the test file 

In [57]:
df_all_test = pd.merge(df_test, df_all, on=['Store','Dept','Date'], how='left')

In [60]:
df_all_test.tail()

,Store,Dept,Date,IsHoliday,Prediction
115059,45,98,2013-06-28,False,1829.142279
115060,45,98,2013-07-05,False,1846.225115
115061,45,98,2013-07-12,False,1867.852846
115062,45,98,2013-07-19,False,1530.992450
115063,45,98,2013-07-26,False,1500.058850


- checking if there is any null (i.e. no prediction from the results files)
- fill with 0

In [69]:
df_all_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 115064 entries, 0 to 115063
Data columns (total 5 columns):
Store         115064 non-null int64
Dept          115064 non-null int64
Date          115064 non-null object
IsHoliday     115064 non-null bool
Prediction    115064 non-null float64
dtypes: bool(1), float64(1), int64(2), object(1)
memory usage: 4.5+ MB


In [67]:
df_all_test['Prediction'] = df_all_test['Prediction'].fillna(0)

In [85]:
#df_all_test[df_all_test['Prediction'].isnull()]
#df_all_test[df_all_test['Prediction'] < 0]

- for the case that prediction < 0, fill 0

In [76]:
df_all_test['Prediction'] = df_all_test['Prediction'].apply(lambda x: 0 if x<0 else x)

## Write the output file

In [83]:
sum(df_all_test['Prediction'] < 0)

0

In [88]:
f_out = open('sub_predictions.csv', 'w')
f_out.write('Id,Weekly_Sales\n')

#for i in range(20):
for i in range(len(df_all_test)):
    out = '{}_{}_{},{}'.format(df_all_test['Store'].iloc[i], 
                                df_all_test['Dept'].iloc[i], 
                                df_all_test['Date'].iloc[i], 
                                round(df_all_test['Prediction'].iloc[i], 2)
                               )
    f_out.write(out + '\n')
f_out.close()